# Alkane Simulation with OpenMM

In [ ]:
# OpenMM imports
import openmm as mm
from openmm import app, unit

## Simulation Parameters

In [ ]:
# Read in our structure and force field
pdb_file = "data/ethane.pdb"
ff_file = "data/ethane.gaff2.xml"

pdb = app.PDBFile(pdb_file)
forcefield = app.ForceField(ff_file)

In [ ]:
# Create OpenMM representation of the system
system = forcefield.createSystem(pdb.topology, nonbondedMethod=app.NoCutoff, constraints=app.HBonds)

In [ ]:
## Set up the integrator
# run gas phase simulation using a Langevin thermostat (integrator)
# at 298.15 K
# with friction constant (or coupling constant) of 5.0 ps^-1
# with 2 fs step size 

# Intergrator parameters
temperature = 298.15*unit.kelvin # temperature in Kelvin
friction_constant = 5.0/unit.picoseconds # coupling constant in picoseconds^-1 
stepsize = 2.0*unit.femtoseconds # in femtoseconds

# Set temperature, coupling constant, and time step
integrator = mm.LangevinIntegrator(temperature, friction_constant, stepsize)

integrator.setConstraintTolerance(1e-5)

In [ ]:
# Initialize the simulation
platform = mm.Platform.getPlatformByName('Reference')
simulation = app.Simulation(pdb.topology, system, integrator, platform)
simulation.context.setPositions(pdb.positions)

# Energy Minimization

The minimization will move the atoms slightly to get them in a more energetically favorable position.

In [ ]:
print('Minimizing...')

st_initial = simulation.context.getState(getPositions=True,getEnergy=True)
print(F"Potential energy before minimization is {st_initial.getPotentialEnergy()}")

simulation.minimizeEnergy(maxIterations=100)

st_minimized = simulation.context.getState(getPositions=True,getEnergy=True)
print(F"Potential energy after minimization is {st_minimized.getPotentialEnergy()}")

# Equilibration

We'll run the simulation for a few time steps to make sure the system is equilibrated. We usually do this at the start of a simulation to make sure the data we collect later isn't influenced by the starting position of the atoms.

In [ ]:
from sys import stdout

print('Equilibrating...')

# Sets up what information OpenMM will report as the simulations runs
simulation.reporters.append(app.StateDataReporter(stdout, 100, step=True, 
    potentialEnergy=True, temperature=True, separator='\t'))

# Set initial velocities
simulation.context.setVelocitiesToTemperature(150.0*unit.kelvin)

simulation.step(2500)

# Production

This is the portion of the simulation we will collect data from and analyze.

In [ ]:
import time

print('Running Production...')

# Begin timer
tinit=time.time()

# Clear simulation reporters
simulation.reporters.clear()

# Reinitialize simulation reporters. 
# We do this because we want different information printed 
# from the production run than the equilibration run.
# Output basic simulation information below every 250000 steps (How many picoseconds is?) 

simulation.reporters.append(app.StateDataReporter(stdout, 250000, 
    step=True, time=True, potentialEnergy=True, temperature=True, 
    speed=True, separator=','))

# write out a trajectory (i.e., coordinates vs. time) to a DCD
# file every 100 steps = 0.2 ps
simulation.reporters.append(app.DCDReporter(F'ethane_sim_{temperature}.dcd', 100))

# run the simulation for 1.0x10^7 steps - 20 ns
simulation.step(10000000)

# End timer
tfinal=time.time()
print('Done!')
print('Time required for simulation:', tfinal-tinit, 'seconds')

### Summary of this worked example

This workflow contains all of the necessary commands to use OpenMM to carry out a gas phase simulation of a single ethane molecule. 

As written, it requires the files:
* ethane.gaff2.xml (force field information)
* ethane.pdb (starting coordinates and topology)
to be imported to the notebook

Briefly, this notebook carried out: 
1. Up to 100 steps of energy minimization using the L-BFGS algorithm.
2. A 5.0 ps MD simulation to bring the ethane molecule to an equilibrium temperature of 298 K (25 °C) in which output is printed every 0.2 ps (100 steps).
3.	A 20 ns MD simulation at 298 K in which output is printed every 500 ps and structures are saved every 0.2 ps into a file called ethane_sim.dcd.  (This trajectory file is written in a commonly used binary format called DCD.)

## Your Turn! - Simulation of Butane

Create a copy of this notebook of the ethane simulation. 
Modify this notebook to:
1. Read in the files **butane.gaff2.xml** and **butane.pdb**
2. Carry out a 10 ps MD simulation to bring the butane molecule to an equilibrium temperature of 298 K in which output is printed every 0.5 ps (Leave the minimization portion beforehand unchanged.)
3. Carry out a 40 ns MD simulation at 298 K in which output is printed every 1 ns and structures are (still) saved every 0.2 ps into a file called butane_sim.dcd